In [4]:
import importlib
import time
import re
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings

import pandas as pd
import tqdm
import util
import uuid

from langchain.text_splitter import RecursiveCharacterTextSplitter

import concurrent.futures

In [2]:
prompt = """
你是一位资深的文字专家，请为以下内容提供用于检索的标题和关键词：

字数要求： 共50字左右 

标题要求： 突出原文的主题和要点，使用简洁明了的语言表达。能概括文档的各个内容，方便用于检索。 关键词突出原文的要点。如果原文没有明显的信息，你的回复是：无。
输出样例：
【标题】：你生成的标题。
【关键词】：你生成的关键词。

以下是原文：
{content}
"""

In [3]:
poets1 = CSVLoader(file_path="4个数据集/poets_3154_copy.csv", encoding="utf-8").load()
poets2 = CSVLoader(file_path="4个数据集/poets_contents_3154.csv", encoding="utf-8").load()

In [5]:
poets1_1 = []
poets2_1 = []

In [29]:
pattern = re.compile(r'诗人: ([\u4e00-\u9fa5]+)\n')

spliter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 64,
    length_function = len,
    is_separator_regex = True,
    separators=["\n\n","\n"," ",""]
)


In [ ]:
for p in tqdm.tqdm(poets2,total=len(poets2)):

    match = pattern.search(p.page_content)
    if match:
        name = match.group(1)
    else:
        print(f"\nnot find name:{p.page_content[0:20]}")
        continue

    head = f"诗人：{name}的生平片段"
    if len(p.page_content)<=500:
        poets2_1.append(head+p.page_content)
    else:
        texts = spliter.split_text(p.page_content)
        for i,text in enumerate(texts):
            poets2_1.append(head+f"{i+1}：\n"+text)

In [38]:
poets2_1[1]

'诗人：方干的生平片段2：\n方干擅长律诗，清润小巧，且多警句。其诗有的反映社会动乱，同情人民疾苦；有的抒发怀才不遇，求名未遂的感怀。文德元年，方干客死会稽，归葬桐江。门人相与论德，谥曰“玄英先生”，并搜集他的遗诗370余篇，编成《方干诗集》传世。《全唐诗》编有方干诗6卷348篇。宋景佑年间，范仲淹守睦州，绘方干像于严陵祠配享。'

In [ ]:
iterObj = poets1
concur = 3
res = dict()
count = 0
chunks = util.chunks(iterObj, batch_size=concur)
for batch in tqdm.tqdm(chunks, total=len(iterObj) // concur):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # 调用你的函数，并传入不同的参数
        # 这里的参数可以是不同的消息、历史记录等
        # 返回的futures列表包含每个调用的结果
        futures = [executor.submit(util.syncChatChatGLM,
                                   prompt.format(content=batch[tag].page_content),
                                   [], 0.95, 0.7, count+tag) for tag in range(concur)]
        for i in range(concur):
	        res[count+i] = {"文档":batch[i].page_content}
        count += concur
        print()
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            try:
                # 获取函数的返回值

                # 关键问题：返回顺序不一定一致！
                
                result = future.result()
                res[result[0]]["摘要"] = result[1]
                print(f"Tag:{result[0]},Result: {result}")
            except Exception as e:
                print(f"Error: {e}")
        time.sleep(1.2)

In [25]:
print(res[2])

{'文档': '诗人: 张旭\n朝代: 唐代\n简介: 张旭（675年—约750年），字伯高，一字季明，汉族，唐朝吴县（今江苏苏州）人。曾官常熟县尉，金吾长史。善草书，性好酒，世称张颠，也是“饮中八仙”之一。其草书当时与李白诗歌、裴旻剑舞并称“三绝”，诗亦别具一格，以七绝见长。与李白、贺知章等人共列饮中八仙之一。唐文宗曾下诏，以李白诗歌、裴旻剑舞、张旭草书为“三绝”。又工诗，与贺知章、张若虚、包融号称“吴中四士”。传世书迹有《肚痛帖》、《古诗四帖》等。', '摘要': ' 【标题】：唐代诗人张旭：草书大家与“饮中八仙”\n【关键词】：张旭，唐代，诗人，草书，饮中八仙，李白，肚痛帖，古诗四帖'}
